In [1]:
import numpy as np
import os, sys
import h5py
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import datetime as DT
import matplotlib.dates as mdates

#sys.path.remove(sys.path[0])
#sys.path.append('/Users/ajc/Core/codes/simulator/debug/ats/tools/utils/')
#sys.path.append('/Users/ajc/Core/codes/simulator/debug/ats/tools/utils/')

import plot_water_table2D

import importlib as imp
print (plot_water_table2D.__file__)
font = {'family' : 'normal',
        'weight' : 'normal',
        'size'   : 12}
matplotlib.rc('font', **font)
import parse_ats
print (parse_ats.__file__)

import get_transect_data

/Users/ajc/Core/codes/simulator/repos/amanzi/src/physics/ats/tools/utils/plot_water_table2D.py
/Users/ajc/Core/codes/simulator/repos/amanzi/src/physics/ats/tools/utils/parse_ats.py


In [2]:
path = "/Users/ajc/Core/simulations/hillslope/rain_advection/SIM_06/"

In [3]:
Simulations = [['S3_P20_%sX'%(i+1) for i in range(1)], ['S3_P20_%sX'%(i+3) for i in range(1)],
              ['S3_P20_%sZ'%(i+1) for i in range(1)], ['S3_P20_%sZ'%(i+3) for i in range(1)],
              ['S3_P20_%sA'%(i+1) for i in range(1)], ['S3_P20_%sA'%(i+3) for i in range(1)],
              ['S3_P20_%sC'%(i+1) for i in range(1)], ['S3_P20_%sC'%(i+3) for i in range(1)]]

Simulations = [['S3_P20_%sA'%(i+1) for i in range(1)], ['S3_P20_%sA'%(i+3) for i in range(1)],
              ['S3_P20_%sB'%(i+1) for i in range(1)], ['S3_P20_%sB'%(i+3) for i in range(1)]]

print (Simulations)
Sim_name = ['S3_P20_1x','S3_P34_1x','S3_P50_1x','S3_P20_3x','S3_P34_3x','S3_P50_3x']
Sim_name = ['S3_P20_1X','S3_P20_3X', 'S3_P20_1Z','S3_P20_3Z','S3_P20_1A','S3_P20_3A','S3_P20_1C','S3_P20_3C']
Sim_name = ['S3_P20_1A','S3_P20_3A', 'S3_P20_1B','S3_P20_3B']

[['S3_P20_1A'], ['S3_P20_3A'], ['S3_P20_1B'], ['S3_P20_3B']]


In [4]:
def getData(i):
    Data = []
    Time = []
    for s in Simulations[i]:
        #print (i, s)
        infile = os.path.join(path,s)
        
        Tim1, Dat1 = get_transect_data.get2DMatrix(infile,varnames=['temperature'])
        a1 = len(Dat1[0,:,0,0])%365
        a2 = len(Dat1[0,:,0,0]) - a1
        
        Tim = Tim1[:a2]
        Dat = Dat1[:,:a2,:,:]
        Time.append(Tim)
        Data.append(Dat)
    d = dict()
    d['temperature'] = Data
    d['time'] = Time
    
    return d

In [5]:
from multiprocessing import Pool
num_processors = 4
p=Pool(processes = num_processors)

#DataS1, DataS2, DataS3, DataS4,DataS5, DataS6,DataS7,DataS8 = p.map(getData,[i for i in range(len(Simulations))])
DataS1, DataS2, DataS3, DataS4 = p.map(getData,[i for i in range(len(Simulations))])

In [6]:
#Data = [DataS1, DataS2, DataS3, DataS4, DataS5, DataS6,DataS7,DataS8]
Data = [DataS1, DataS2, DataS3, DataS4]

In [236]:
def get_temp_at_depth(dat,season = " ", start=0, end=-1,depth_s=-1,depth_e=-1, yr_cyc=365,domain='entire'):
    #25 * 2cm = 50 cm
    sim_len = len(dat['temperature'])
    x_pos = start
    data = []
    times = []
    HillslopeT = dict()
    AvgTemp = [] # np.zeros(cycles)
    for k in range(sim_len):
        data = np.array(dat['temperature'][k])
        time = dat['time'][k]
        nvar, cycles, xnum, znum = data.shape
     
        zlen = len(data[1,0,0,:])-1
        xlen = len(data[0,0,:,0])-1
       
        for cyc in range(cycles):
            sum1 = data[nvar-1,cyc,x_pos,zlen-depth_s]
            #print ('S: ',sum1, zlen, depth_s)
            AvgTemp.append(sum1)
        times.append(time)
    L = len(AvgTemp)%int(yr_cyc)
    times = np.concatenate(times)
    #print ('Lengths: ', len(AvgTemp), len(times))
    HillslopeT['times'] = np.array([t*365 - 365. for t in times])
    HillslopeT_Avg = np.array(AvgTemp)
    
    HillslopeT['temp'] = HillslopeT_Avg
    return HillslopeT



In [104]:
depth1 = [[0,10]] # active layer, 0.5-2 m, 2-3.5m

In [105]:
start1_,end1_, = 100,5
Data_X = dict()

permafrost = ['top']
Names = ['S1', 'S2','S3', 'S4','S5', 'S6','S7', 'S8']
for i in range(len(permafrost)):
    for dat,name in zip(Data,Names):
        #print (name)
        d = get_temp_at_depth(dat,start=start1_,end=end1_,depth_s=depth1[i][1],depth_e=depth1[i][0],yr_cyc=365,domain='entire')
        Data_X[name + '_temp'] = d['temp']
        Data_X[name +'_time'] = d['times']
        #break

Lengths:  365 365
Lengths:  365 365
Lengths:  365 365
Lengths:  365 365


In [168]:
Data_X.keys()
#print (Data_X['S1_top_temp'])
outfile='/Users/ajc/Documents/MyPapers/RainWarming/figures/'

In [254]:
# Three zones : Active layer, shallow permafrost, deep permafrost
%matplotlib qt

def Plot_Wetlands_Uplands():
    fig, ax = plt.subplots(1,2, figsize=(6,4), facecolor='w', edgecolor='k',gridspec_kw={'width_ratios':[1,1]})
    fig.subplots_adjust(hspace =.02, wspace=1.02)
    TempC = 1*273.15
    plt.tight_layout()
    ax = ax.ravel()
    xpos = np.array([0, 1,2,3])
    Xticks = [r'S$_{0}$', r'S$_{30}$', r'S$_{60}$', r'S$_{90}$']
    Colors = ['k','r','g','c']
    marks = ['solid', 'dashed', 'dotted']
    X, T1 = [], []

    start1_,end1_, = 100,5
    

    Names = ['S1', 'S2','S3', 'S4','S5', 'S6','S7', 'S8']
    
    for k,lenx in enumerate([50,100]):
        axs = ax[k]
        for i,dp in enumerate([5,10,15]):
            Data_X = dict()
            for dat,name in zip(Data,Names):

                d = get_temp_at_depth(dat,start=lenx,end=100,depth_s=dp,depth_e=0,yr_cyc=365,domain='entire')
                Data_X[name + '_temp'] = d['temp']
                Data_X[name +'_time'] = d['times']
            #print (Data_X.keys())
            S1 = [x - TempC for x in Data_X['S1_temp']]
            S2 = [x - TempC for x in Data_X['S2_temp']]
            T = Data_X['S1_time']
            DX =  np.array(S2) - np.array(S1)
            if i ==0:
                axs.plot(T,DX,color=Colors[0],label='Wetter soil',linestyle=marks[i],linewidth=1.)
            else:
                axs.plot(T,DX,color=Colors[0],linestyle=marks[i],linewidth=1.)

            S1 = [x - TempC for x in Data_X['S3_temp']]
            S2 = [x - TempC for x in Data_X['S4_temp']]
            T = Data_X['S3_time']
            DX =  np.array(S2) - np.array(S1)
            if i ==0:
                axs.plot(T,DX,color=Colors[1],label='Drier soil',linestyle=marks[i],linewidth=1.)
            else:
                axs.plot(T,DX,color=Colors[1],linestyle=marks[i],linewidth=1.)

            
        
        axs.set_xlabel('Time [day]',fontsize=12, fontweight='normal')
        axs.set_xlim(233,260)
        axs.set_ylim(-0.25,.25)
        #axs.axvline(235,ymin=0.3, ymax=0.7,color='b',linestyle='dashed')
        #axs.axvline(260,ymin=0.3, ymax=0.7,color='b',linestyle='dashed')
        axs.text(234,-0.225,'$x$ = %s m'%lenx,fontsize=12, fontweight='normal')
    ax[0].legend(loc='upper left',fontsize=8,labelspacing=0.5)
    ax[0].set_ylabel(u'$\Delta$ T [\u00B0C]',fontsize=12, fontweight='normal')
    plt.tight_layout()
    plt.savefig(outfile+'temp_at_depths-50m-100m.jpeg', bbox_inches='tight', dpi=300)
    plt.show()



In [255]:
Plot_Wetlands_Uplands()